In [12]:
import numpy as np
import cvxpy as cp
import matplotlib.pyplot as plt
import math
import itertools

## Multivariable

In [13]:
# Generate 100 2d random points
np.random.seed(0)
n = 100
d = 2
X = np.random.multivariate_normal(mean=np.zeros(d), cov=np.eye(d), size=n)

In [14]:
# This is result from chatgpt. Needed to be verified
def wedge_product(vectors):
    """
    Compute the wedge product of multiple d-dimensional vectors.
    
    Args:
        vectors (list of np.array): List of k d-dimensional vectors.
    
    Returns:
        float: Magnitude of the wedge product.
    """
    # Stack vectors into a matrix
    M = np.stack(vectors, axis=0)  # k x d matrix
    
    # Check dimensions
    k, d = M.shape
    if k > d:
        raise ValueError("Cannot compute wedge product: more vectors than dimensions.")
    
    # Compute the Gram matrix
    G = np.dot(M, M.T)  # k x k Gram matrix
    
    # Compute the volume (magnitude of the wedge product)
    volume = np.sqrt(np.linalg.det(G))
    return volume

In [15]:
# find K matrix from the given data
def find_K(X: np.ndarray) -> np.ndarray:
    n, d = X.shape
    c = math.comb(n, d-1)

    K = np.zeros((n, c))
    row_idx = np.arange(n)

    # Generate possible combinations of d-1 elements from n elements
    idx_combinations = list(itertools.combinations(row_idx, d-1))

    for i in range(n):
        x_i = X[i]
        for j in range(c):
            x_j = X[idx_combinations[j]]
            
            # Compute outer product of every row in x_j and take L1 norm
            denominator = np.linalg.norm(x_j, ord=1)

            numerator = np.linalg.det(np.outer(x_i, x_j))
            numerator = np.maximum(numerator, 0)

            K[i, j] = numerator / denominator

    return K

K = find_K(X)
print(K.shape)

(100, 100)


In [ ]:
# L is label matrix
epsilon = 0.1
L = np.random.multivariate_normal(mean=np.zeros(d), cov=epsilon*np.eye(d), size=n)
L = L / epsilon

In [ ]:
c = math.comb(n, d-1)
reg_lambda = 1
Z = cp.Variable((c, n))

# Define the optimization problem
regularization_term = reg_lambda * cp.sum([cp.norm(Z[:, i], 2) for i in range(n)])
objective = cp.Minimize(cp.norm(K.T @ Z - L, 'fro')**2 + regularization_term)

# Solve the optimization problem
problem = cp.Problem(objective)
problem.solve()

# Extract the optimal value and optimal Z
optimal_value = problem.value
optimal_Z = Z.value

In [ ]:
# Achieve NN
# They fucked up the notation in Eq 34..., 
# We should look Eq 48 of arxiv.org/pdf/2309.16512 
def NN(x: np.ndarray) -> np.ndarray:
    for j in range(n):
